In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

In [ ]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

In [ ]:
from azureml.core import Dataset
# Get a dataset by name and version number
anji_data_ds = Dataset.get_by_name(workspace = ws,
                                 name = 'anji_data', 
                                 version = 1)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "GPUsCluster"

compute_target = ComputeTarget(workspace=ws, name=cluster_name)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

In [ ]:
from azureml.core import Experiment

experiment_name = 'exp-yolov3-poc'
experiment = Experiment(ws, name=experiment_name)

In [ ]:
from azureml.core import Environment

pytorch_env = Environment.from_conda_specification(name = 'pytorch-1.6-gpu', file_path = './conda_dependencies.yml')

# Specify a GPU base image
#pytorch_env.docker.enabled = True
#pytorch_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04'

#dockerfile = """
#FROM mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04
#RUN apt update
#RUN apt-get -y install sudo
#RUN sudo apt -y install libgl1-mesa-glx
#"""
#pytorch_env.docker.enabled = True
#pytorch_env.docker.base_image = None
#pytorch_env.docker.base_dockerfile = dockerfile

In [ ]:
from azureml.core import ScriptRunConfig

project_folder = '.'

src = ScriptRunConfig(source_directory=project_folder,
                      script='pytorch_train.py',
                      #arguments=['--num_epochs', 30, '--output_dir', './outputs'],
                      arguments=['--epochs', 2,'--data-folder', anji_data_ds.as_mount('/tmp/tmp_anjidata')],
                      #compute_target=compute_target,
                      environment=pytorch_env)

In [ ]:
run = experiment.submit(src,tags={"purpose":"test metrix logging"})
print(run)

In [ ]:
run.wait_for_completion(show_output=True)

In [ ]:
assert(run.get_status() == "Completed")

In [ ]:
print(run.get_file_names())

In [ ]:
from azureml.core import Run
run = Run(experiment, 'exp-yolov3-poc_1608263349_9b92b559')

In [ ]:
model = run.register_model(model_name = 'yolov3-poc-model', model_path = 'outputs/best.pt')
print(model.name, model.id, model.version, sep = '\t')

In [ ]:
from azureml.core import Environment
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice
from azureml.core.model import Model

model = Model(ws, 'yolov3-poc-model')

pytorch_env = Environment.from_conda_specification(name = 'pytorch-1.6-gpu', file_path = './conda_dependencies.yml')
dockerfile = """
FROM mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04
RUN apt update
RUN apt-get -y install sudo
RUN sudo apt -y install libgl1-mesa-glx
"""
pytorch_env.docker.enabled = True
pytorch_env.docker.base_image = None
pytorch_env.docker.base_dockerfile = dockerfile
pytorch_env.inferencing_stack_version = 'latest'

inference_config = InferenceConfig(
        entry_script="pytorch_score.py", 
        environment=pytorch_env,
        source_directory = "./"
        )

aciconfig = AciWebservice.deploy_configuration(cpu_cores=2, 
                                               memory_gb=2, 
                                               tags={'framework':'pytorch'},
                                               description='Learning with PyTorch')

service = Model.deploy(workspace=ws, 
                           name='aci-yolov3-poc', 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig)
service.wait_for_deployment(True)
print(service.state)

In [ ]:
service.get_logs()

In [ ]:
print(service.scoring_uri)

In [ ]:
import json
from PIL import Image
import matplotlib.pyplot as plt

%matplotlib inline
plt.imshow(Image.open('testfiles/test-image-2.jpg'))

In [ ]:
import requests
from base64 import b64encode
# send a random row from the test set to score
with open('testfiles/test-image-1.jpg', 'rb') as jpg_file:
    byte_content = jpg_file.read()
    jpg_file.close()
base64_bytes = b64encode(byte_content)
base64_string = base64_bytes.decode()

#input_data = "{\"data\": " + base64_string + "}"
input_data=json.dumps({'data': base64_string})
#print(input_data)
headers = {'Content-Type':'application/json'}

# for AKS deployment you'd need to the service key in the header as well
# api_key = service.get_key()
# headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ api_key)} 

resp = requests.post(service.scoring_uri, input_data, headers=headers)

#print(resp.text)


In [ ]:
print(resp.text)

In [ ]:

max_len = len(resp.text) -1
result = resp.text[1:max_len].replace("\\","")

In [ ]:
import base64

result_image = json.loads(result)["result_image"]

image_data = base64.b64decode(result_image)

with open('testfiles/test-impage-1-detection.jpg', 'wb') as jpg_file:
    jpg_file.write(image_data)

%matplotlib inline
plt.imshow(Image.open('testfiles/test-impage-1-detection.jpg'))